In [1]:
# magic
#################################
%matplotlib inline 
#################################

# run magic в юпитере! НЕ РАБОТАЕТ в .py файлах
# run files исполнить файлы магией тут
%run /srv/jhub/share/._Additions/import_my_modules.py

# %run /srv/jhub/share/._Additions/import_libs.py
# %run /srv/jhub/share/._Additions/prints_.py
# %run /srv/jhub/share/._Additions/my_script.py этот импортится внутри import_my_modules.py
%run /srv/jhub/share/._Additions/Connecting_.py
%run /srv/jhub/share/._Additions/Date_gen.py
%run /srv/jhub/share/._Additions/xx.py

#################################
# еще вариант magic который !РАБОТАЕТ в .py файлах
# get_ipython().run_line_magic('matplotlib', 'inline')
# run files
# get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/import_libs.py')
# get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/Connecting_.py')
# get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/my_script.py')
# get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/Date_gen.py')
# get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/xx.py')
############################## End ##############################
# help(Prints_), print(JiraTaskPrint.__doc__)

In [2]:
# host

# Среднесуточные складские остатки в разрезе индекса продаж

# Для обновления данных нажмите Alt+Ctrl+L 
(раскладка клавиатуры - ENG)

In [3]:
# выводим спарсенные данные с жиры описание и тд....
task_link = '''
https://jira.e2e4.ru/browse/BA-\
380
'''
JiraTaskPrint().print_tasks(task_link)

Данные с jira 
(подтягиваются автоматически и обнавляются при запуске)


HTML(value="<a href='\nhttps://jira.e2e4.ru/browse/BA-380\n'>\nhttps://jira.e2e4.ru/browse/BA-380\n</a>")

Бизнес аналитика / BA-380 

Добавить в отчет "Среднесуточные складские остатки в разрезе индекса продаж" возможность выбора продакта 


Описание: 

есть отчет, показывающий информацию в разрезе индекса продаж.
https://confluence.e2e4.ru/pages/viewpage.action?pageId=183323588&run_11_from=01.06.2022&run_11_til=30.06.2022&run_11_storage=0&run_11_category=347&run_11_brand=&run_11=run
здесь есть выбор Склада, Категории, Бренда.
Нужно добавить возможность выбора продакта (какого-то конкретного или всех), чтобы можно было снимать отчет по всем товарам продакта (или по всем товарам, если выбрано значение "все продакты")
список продактов есть вот в этом отчете:
https://confluence.e2e4.ru/pages/viewpage.action?pageId=188978207&run_1_start=27.06.2022&run_1=run
в итоге примерно вот так это должно будет выглядеть
  

            


In [4]:
#  КОПИЯ РАБОЧАЯ !!!
# sql_query_orderstat_site = ''' 

#     WITH params AS (
#         SELECT
#             'DATE_START_replce'::date AS frm, 
#             'DATE_END_replce'::date AS til,
#             STORAGE_replce AS storage,
#             CATEGORY_replce AS category,
#             'BRAND_replce' as brand
#             --DEL_IN_USE_replce as in_use
#             --PRODUCTMANAGER_replce 
#     )
#     , items AS (
#         SELECT i.id, i.name, i."rank", i.id_category
#         --    	,_.frm    	,_.til    	,(1 + _.til - _.frm)
#         FROM params _
#         CROSS JOIN catalog.items i
#         JOIN sales.items s ON s.id_catalog_item = i.id
#         JOIN "catalog".categories c ON c.id = i.id_category
#         LEFT JOIN "catalog".brands b ON b.id = i.id_brand
#         WHERE 
#             s.sold_at BETWEEN _.frm AND _.til
#             and i.RANK IS NOT NULL
#             AND CASE WHEN _.category = 0 THEN TRUE ELSE i.id_category = _.category END
#             AND CASE WHEN _.brand = '' THEN TRUE ELSE b.name = _.brand END
#     )
#     , categories AS (
#         SELECT DISTINCT c.id, c.name, i."rank", c."order", pu.name "suppliers_name"
#         FROM items i
#         JOIN "catalog".categories c ON c.id = i.id_category
#         left join public.users pu on c.id_product_manager = pu.id 
#         where 
#             pu.fired_at is null
            
            
#             and case when 0 = 0 then c.id_product_manager in (select cc.id_product_manager from catalog.categories cc join public.users pu on cc.id_product_manager = pu.id where pu.fired_at is null)
#                 else c.id_product_manager = 0 end

#     )
#     , sales AS (
#         SELECT
#             i.id_category,
#             i."rank",
#             count(DISTINCT i.id) AS count,
#             count(s.id)::numeric / (1 + _.til - _.frm) AS quantity,
#             sum(s."cost")::numeric / (1 + _.til - _.frm) AS sum
#         FROM params _
#         CROSS JOIN items i
#         JOIN sales.items s ON s.id_catalog_item = i.id
#         JOIN public.storage st ON st.id = s.id_storage
#         WHERE s.sold_at BETWEEN _.frm AND _.til
#             AND CASE WHEN _.storage = 0 THEN TRUE ELSE COALESCE(st.parentstorageid, st.id) = _.storage END
#         GROUP BY _.frm, _.til, i.id_category, i."rank"
#     )
#     , stock AS (
#         SELECT
#             i.id_category,
#             i."rank",
#             sum(h.stock_free
#                 + h.stock_reserved
#                 + h.relocation_free
#                 + h.relocation_reserved
#                 + h.arrivals_free
#                 + h.arrivals_reserved
#                 + h.transit_free
#                 + h.transit_reserved)::numeric / (1 + _.til - _.frm) AS quantity,
#             sum(h.stock_free_sum
#                 + h.stock_reserved_sum
#                 + h.relocation_free_sum
#                 + h.relocation_reserved_sum
#                 + h.arrivals_free_sum
#                 + h.arrivals_reserved_sum
#                 + h.transit_free_sum
#                 + h.transit_reserved_sum)::numeric / (1 + _.til - _.frm) AS sum,
#             sum(h.stock_free
#                 + h.relocation_free
#                 + h.arrivals_free
#                 + h.transit_free
#             )::numeric / (1 + _.til - _.frm) AS quantity_free,
#             sum(h.stock_free_sum
#                 + h.relocation_free_sum
#                 + h.arrivals_free_sum
#                 + h.transit_free_sum
#             )::numeric / (1 + _.til - _.frm) AS sum_free
#         FROM params _
#         CROSS JOIN items i
#         JOIN store.stock_history h ON h.id_reference_item = i.id
#         JOIN public.storage st ON st.id = h.id_storage
#         WHERE h."date" BETWEEN _.frm AND _.til
#             AND CASE WHEN _.storage = 0 THEN TRUE ELSE COALESCE(st.parentstorageid, st.id) = _.storage END
#         GROUP BY _.frm, _.til, i.id_category, i."rank"
#     )  
#     , EA as (
#     SELECT
#          c.suppliers_name "Продакт",
#         --c.name "Категория",
#         c.RANK "Индекс продаж",
#         case when count(sales.count) = 0 then null else count(sales.count) end "Продажи уникальных артикулов, шт",
#         case when count(round(sales.quantity, 2)) = 0 then null else count(round(sales.quantity, 2)) end "СС Продажи, шт",
#         case when sum(sales.sum::int) = 0 then null else sum(sales.sum::int) end "СС Продажи, руб (себестоимость)",
#         case when count(round(stock.quantity, 2)) = 0 then null else count(round(stock.quantity, 2)) end "СС Остатки, шт",
#         case when sum(stock.sum::int) = 0 then null else sum(stock.sum::int) end "СС Остатки, руб (себестоимость)",
#         case when count(round(stock.quantity_free, 2)) = 0 then null else count(round(stock.quantity_free, 2)) end "СС Остатки-свободный, шт",
#         case when sum(stock.sum_free::int) = 0 then null else sum(stock.sum_free::int) end "СС Остатки-свободный, руб (себестоимость)",
#         CASE WHEN count(sales.quantity)=0 THEN NULL ELSE count(round(stock.quantity / sales.quantity)) END AS "Оборачиваемость (шт/шт)",
#         CASE WHEN sum(sales.sum)=0 THEN NULL ELSE sum(round(stock.sum / sales.sum)) END AS "Оборачиваемость (руб/руб)"
#     FROM categories c
#     LEFT JOIN sales ON sales.id_category = c.id AND sales.RANK = c.rank
#     LEFT JOIN stock ON stock.id_category = c.id AND stock.RANK = c.RANK
#     WHERE 
#         GREATEST(sales.quantity, sales.sum, stock.quantity, stock.sum) > 0
#     group by c.suppliers_name, c.rank 
#     ORDER BY c.suppliers_name, c.rank --c.order,
#     )

#     select  *
#     --	"Продакт",
#     --	"Индекс продаж","Продажи уникальных артикулов, шт","СС Продажи, шт","СС Продажи, руб (себестоимость)","СС Остатки, шт",
#     --	"СС Остатки, руб (себестоимость)","СС Остатки-свободный, шт","СС Остатки-свободный, руб (себесто","Оборачиваемость (шт/шт)","Оборачиваемость (руб/руб)"
#     from EA


#      '''

In [5]:
# КОПИЯ РАБОЧАЯ но не павильно))
# sql_query_orderstat_site = ''' 

#     WITH params AS (
#         SELECT
#             'DATE_START_replce'::date AS frm, 
#             'DATE_END_replce'::date AS til,
#             STORAGE_replce AS storage,
#             CATEGORY_replce AS category,
#             'BRAND_replce' as brand
#             --DEL_IN_USE_replce as in_use
#             --PRODUCTMANAGER_replce 
#             --DEL_STORAGE_NSK_replce 
#     )
#     , items as (
#         select i.id, i.name, i.rank, i.id_category
#         --    	,_.frm    	,_.til    	,(1 + _.til - _.frm)
#         from params _
#         cross join catalog.items i
#         join catalog.categories c on c.id = i.id_category
#         left join catalog.brands b on b.id = i.id_brand
#         where 
#             i.rank is not null
#             and case when _.category = 0 then true else i.id_category = _.category end
#             and case when _.brand = '' then true else b.name = _.brand end
            
#             and case when DEL_IN_USE_replce = 1 then true else c.name not ilike 'уцененные товары%' and c.name not ilike 'внутреннее использование%' end
#         -- and case when 0 = 1 then true else c.name not ilike 'уцененные товары%' and c.name not ilike 'внутреннее использование%' end

#     )
#     , categories as (
#         select distinct c.id, c.name, i.rank, c.order, pu.name "suppliers_name"
#         from items i
#         join catalog.categories c on c.id = i.id_category
#         left join public.users pu on c.id_product_manager = pu.id 
#         where 
#         --    pu.fired_at is null
#         --    and 
#             case when PRODUCTMANAGER_replce = 0 then true
#         --    c.id_product_manager in (select cc.id_product_manager from catalog.categories cc join public.users pu on cc.id_product_manager = pu.id where pu.fired_at is null)
#                 else c.id_product_manager = PRODUCTMANAGER_replce end
#         --    and 
#         --    case when 0 = 0 then true
#         --    c.id_product_manager in (select cc.id_product_manager from catalog.categories cc 
#         --    	join public.users pu on cc.id_product_manager = pu.id 	
#         --    	where pu.fired_at is null
#         --    	)
#         --        else c.id_product_manager = 424 end
                
#                 and case when DEL_IN_USE_replce = 1 then true else c.name not ilike 'Уцененные товары%' and c.name not ilike 'Внутреннее использование%' end
#         --			and case when 0 = 1 then true else c.name not ilike 'Уцененные товары%' and c.name not ilike 'Внутреннее использование%' end

#     )
#     , sales as (
#         select
#             i.id_category,
#             i.rank,
#             count(distinct i.id) as count,
#             count(s.id)::numeric / (1 + _.til - _.frm) as quantity,
#             sum(s.cost)::numeric / (1 + _.til - _.frm) as sum
#         from params _
#         cross join items i
#         join sales.items s on s.id_catalog_item = i.id
#         join public.storage st on st.id = s.id_storage
#         where s.sold_at between _.frm and _.til
#             and case when _.storage = 0 then true else coalesce(st.parentstorageid, st.id) = _.storage end
#         group by _.frm, _.til, i.id_category, i.rank
#     )
#     , stock as (
#         select
#             i.id_category,
#             i.rank,
#             sum(h.stock_free
#                 + h.stock_reserved
#                 + h.relocation_free
#                 + h.relocation_reserved
#                 + h.arrivals_free
#                 + h.arrivals_reserved
#                 + h.transit_free
#                 + h.transit_reserved)::numeric / (1 + _.til - _.frm) as quantity,
#             sum(h.stock_free_sum
#                 + h.stock_reserved_sum
                
#                 + h.relocation_free_sum
#                 + h.relocation_reserved_sum
#                 + h.arrivals_free_sum
#                 + h.arrivals_reserved_sum
#                 + h.transit_free_sum
#                 + h.transit_reserved_sum)::numeric / (1 + _.til - _.frm) as sum,
#             sum(h.stock_free
#                 + h.relocation_free
#                 + h.arrivals_free
#                 + h.transit_free
#             )::numeric / (1 + _.til - _.frm) as quantity_free,
#             sum(h.stock_free_sum
#                 + h.relocation_free_sum
#                 + h.arrivals_free_sum
#                 + h.transit_free_sum
#             )::numeric / (1 + _.til - _.frm) as sum_free
#         from params _
#         cross join items i
#         join store.stock_history h on h.id_reference_item = i.id
#         join public.storage st on st.id = h.id_storage
#         where h.date between _.frm and _.til
#             and case when _.storage = 0 then true else coalesce(st.parentstorageid, st.id) = _.storage end
            
#             and case when DEL_STORAGE_NSK_replce = 0 then true else st.id not in ( DEL_STORAGE_NSK_replce ) end
#     --            and case when 0 = 1 then true else st.id not in ( 47 ) end
        
#         group by _.frm, _.til, i.id_category, i.rank

#     )  
    
#     , ea as (
#     select
#         c.suppliers_name "Продакт",
#         --    c.name "Категория",
#         c.RANK "Индекс продаж",
#         sum(sales.count) "Продажи уникальных артикулов, шт",
#         sum(round(sales.quantity, 2)) "СС Продажи, шт",
#         sum(sales.sum::int) "СС Продажи, руб (себестоимость)",
#         sum(round(stock.quantity, 2)) "СС Остатки, шт",
#         sum(stock.sum::int) "СС Остатки, руб (себестоимость)",
#         sum(round(stock.quantity_free, 2)) "СС Остатки-свободный, шт",
#         sum(stock.sum_free::int) "СС Остатки-свободный, руб (себестоимость)",
#         CASE WHEN sum(sales.quantity)=0 THEN NULL ELSE sum(round(stock.quantity / sales.quantity)) END AS "Оборачиваемость (шт/шт)",
#         CASE WHEN sum(sales.sum)=0 THEN NULL ELSE sum(round(stock.sum / sales.sum)) END AS "Оборачиваемость (руб/руб)"
#     FROM categories c
#     LEFT JOIN sales ON sales.id_category = c.id AND sales.RANK = c.rank
#     LEFT JOIN stock ON stock.id_category = c.id AND stock.RANK = c.RANK
#     WHERE GREATEST(sales.quantity, sales.sum, stock.quantity, stock.sum) > 0
#     group by c.suppliers_name, c.RANK
#     ORDER BY c.suppliers_name, c.rank --c."order", c.name, c.rank

#     )

#     select  *
#     --	"Продакт",
#     --	"Индекс продаж","Продажи уникальных артикулов, шт","СС Продажи, шт","СС Продажи, руб (себестоимость)","СС Остатки, шт",
#     --	"СС Остатки, руб (себестоимость)","СС Остатки-свободный, шт","СС Остатки-свободный, руб (себесто","Оборачиваемость (шт/шт)","Оборачиваемость (руб/руб)"
#     from EA
    
#     --select 
#     --sum( "СС Остатки, руб (себестоимость)" )
#     --,sum( "СС Остатки-свободный, руб (себестоимость)" )
#     --from EA


#      '''

In [6]:

sql_query_orderstat_site = ''' 
    WITH params AS (
    select
        /*jhub*/
            'DATE_START_replce'::date AS frm, 
            'DATE_END_replce'::date AS til,
            STORAGE_replce AS storage,
            CATEGORY_replce AS category,
            'BRAND_replce' as brand
            --DEL_IN_USE_replce as in_use
            --PRODUCTMANAGER_replce 
            --DEL_STORAGE_NSK_replce 

        /*confluence*/
    --        '\$from'::date AS frm,
    --        '\$til'::date AS til,
    --		\$storage AS storage,
    --		\$category AS category,
    --        '\$brand' AS brand

        /*tests*/
    --        '01.08.2022'::date AS frm, 
    --        '19.08.2022'::date AS til,
    --        0 AS storage,
    --        0 AS category,
    --        '' as brand
    )
	, items AS (
	    SELECT i.id, i.name, i."rank", i.id_category
	    FROM params _
	    CROSS JOIN "catalog".items i
		JOIN "catalog".categories c ON c.id = i.id_category
		LEFT JOIN "catalog".brands b ON b.id = i.id_brand
	    WHERE i.RANK IS NOT NULL
	        AND CASE WHEN _.category = 0 THEN TRUE ELSE i.id_category = _.category END
	        AND CASE WHEN _.brand = '' THEN TRUE ELSE b.name ilike _.brand end
	        
	--        and case when 1 = 1 then true else c.name not ilike 'уцененные товары%' and c.name not ilike 'внутреннее использование%' end
	        /*jhub*/
	        and case when DEL_IN_USE_replce = 1 then true else c.name not ilike 'уцененные товары%' and c.name not ilike 'внутреннее использование%' end
	
	)
	, categories AS (
	    select distinct c.id, c.name, i.rank, c.order, pu.name "suppliers_name"
	    FROM items i
	    JOIN "catalog".categories c ON c.id = i.id_category
	    left join public.users pu on c.id_product_manager = pu.id 
	    where 
	--	    case when 0 = 0 then true
	--	        else c.id_product_manager = 0 end
	--		and case when 1 = 1 then true 
	--			else c.name not ilike 'Уцененные товары%' and c.name not ilike 'Внутреннее использование%' end
	        /*jhub*/
		    case when PRODUCTMANAGER_replce = 0 then true
		        else c.id_product_manager = PRODUCTMANAGER_replce end
		    and case when DEL_IN_USE_replce = 1 then true 
				else c.name not ilike 'Уцененные товары%' and c.name not ilike 'Внутреннее использование%' end  
	)
	, sales AS (
	    SELECT
	        i.id_category,
	        i."rank",
			count(DISTINCT i.id) AS count,
			case when (count(s.id)::numeric / (1 + _.til - _.frm)) = 0 then null 
				else count(s.id)::numeric / (1 + _.til - _.frm) end AS quantity,
	        case when (sum(s."cost")::numeric / (1 + _.til - _.frm)) = 0 then null 
	        	else sum(s."cost")::numeric / (1 + _.til - _.frm) end AS sum
	    FROM params _
	    CROSS JOIN items i
		JOIN sales.items s ON s.id_catalog_item = i.id
		JOIN public.storage st ON st.id = s.id_storage
		WHERE s.sold_at BETWEEN _.frm AND _.til
			AND CASE WHEN _.storage = 0 THEN TRUE ELSE COALESCE(st.parentstorageid, st.id) = _.storage END
		GROUP BY _.frm, _.til, i.id_category, i."rank"
	)
	, stock AS (
	    SELECT
	        i.id_category,
	        i."rank",
			sum(h.stock_free
				+ h.stock_reserved
				+ h.relocation_free
				+ h.relocation_reserved
				+ h.arrivals_free
				+ h.arrivals_reserved
				+ h.transit_free
				+ h.transit_reserved)::numeric / (1 + _.til - _.frm) AS quantity,
	        sum(h.stock_free_sum
				+ h.stock_reserved_sum
				+ h.relocation_free_sum
				+ h.relocation_reserved_sum
				+ h.arrivals_free_sum
				+ h.arrivals_reserved_sum
				+ h.transit_free_sum
				+ h.transit_reserved_sum)::numeric / (1 + _.til - _.frm) AS sum,
	        sum(h.stock_free
				+ h.relocation_free
				+ h.arrivals_free
				+ h.transit_free
	        )::numeric / (1 + _.til - _.frm) AS quantity_free,
	        sum(h.stock_free_sum
				+ h.relocation_free_sum
				+ h.arrivals_free_sum
				+ h.transit_free_sum
	        )::numeric / (1 + _.til - _.frm) AS sum_free
	    FROM params _
	    CROSS JOIN items i
	    JOIN store.stock_history h ON h.id_reference_item = i.id
		JOIN public.storage st ON st.id = h.id_storage
		WHERE 
			h."date" BETWEEN _.frm AND _.til
			AND CASE WHEN _.storage = 0 THEN TRUE 
				ELSE COALESCE(st.parentstorageid, st.id) = _.storage end
				
	--		and case when 1 = 1 then true 
	--			else st.id not in ( 47 ) end
			/*jhub*/
			and case when DEL_STORAGE_NSK_replce = 0 then true 
				else st.id not in ( DEL_STORAGE_NSK_replce ) end
	
		GROUP BY _.frm, _.til, i.id_category, i."rank"
	)
	
	, EA as (
	select
	    c.suppliers_name "Продакт",
	--    c.name "Категория",
	    c.RANK "Индекс продаж",
	    sum(sales.count) "Продажи уникальных артикулов, шт",
	    sum(round(sales.quantity, 2)) "СС Продажи, шт",
	    sum(sales.sum::int) "СС Продажи, руб (себестоимость)",
	    sum(round(stock.quantity, 2)) "СС Остатки, шт",
	    sum(stock.sum::int) "СС Остатки, руб (себестоимость)",
	    sum(round(stock.quantity_free, 2)) "СС Остатки-свободный, шт",
	    sum(stock.sum_free::int) "СС Остатки-свободный, руб (себестоимость)",
	    CASE WHEN sum(sales.quantity)=0 THEN NULL ELSE sum(round(stock.quantity / sales.quantity)) END AS "Оборачиваемость (шт/шт)",
	    CASE WHEN sum(sales.sum)=0 THEN NULL ELSE sum(round(stock.sum / sales.sum)) END AS "Оборачиваемость (руб/руб)"
	FROM categories c
	LEFT JOIN sales ON sales.id_category = c.id AND sales.RANK = c.rank
	LEFT JOIN stock ON stock.id_category = c.id AND stock.RANK = c.RANK
	WHERE 
		GREATEST(sales.quantity, sales.sum, stock.quantity, stock.sum) > 0
	group by  
		c.suppliers_name, c.rank --c.suppliers_name,
	ORDER BY 
		c.rank --c."order", c.name, 
	)
	
    select * from EA

    union all
    select 
        'ИТОГО',
        'ИТОГО', 
        sum("Продажи уникальных артикулов, шт") "Продажи уникальных артикулов, шт"
        , sum("СС Продажи, шт") "СС Продажи, шт"
        , sum("СС Продажи, руб (себестоимость)") "СС Продажи, руб (себестоимость)"
        , sum("СС Остатки, шт") "СС Остатки, шт"
        , sum("СС Остатки, руб (себестоимость)") "СС Остатки, руб (себестоимость)"
        , sum("СС Остатки-свободный, шт") "СС Остатки-свободный, шт"
        , sum("СС Остатки-свободный, руб (себесто") "СС Остатки-свободный, руб (себесто"
        , sum("Оборачиваемость (шт/шт)") "Оборачиваемость (шт/шт)"
        , sum("Оборачиваемость (руб/руб)") "Оборачиваемость (руб/руб)"
    from EA

    --union all
    --select 
    --	'ИТОГО',
    --	'конфа', 13863,	3677.25,	10614289,	359909.52,	667188106,	319112.76,	523616688,	189068,	261366

    /*
    --union all
    --select 
    --	'jhub',  13618,	3661.59,	10521794,	342445.47,	649022085,	306226.57,	509404185,	181383,	251725
    */



     '''

In [7]:
print( colored("Для начала выберите параметры", "blue", attrs=["bold"] ) )

# ДАТЫ
DATE_START_replce = date.today() - relativedelta(months=1)
DATE_END_replce = date.today()
# Класс datetime.datetime принимает аргументы: год, месяц, день, час, минута, секунда и микросекунда.
DatePicker1 = widgets.DatePicker( description='От', disabled=False, value = DATE_START_replce ) # datetime(2017, 4, 21, 0, 0, 0)
DatePicker2 = widgets.DatePicker( description='До', disabled=False, value = DATE_END_replce )

with psycopg2.connect(dbname=database, user=user, password=password, host=host) as cnxn:
# СКЛАДЫ выпадающий список
    df_1 = pd.read_sql_query(
        '''\
        with temp1 as (
        SELECT  
            id "1",  name "2"
        FROM opentech.storage WHERE deleted=0 AND store=1 ORDER BY name
        )
        select 0 "1",  'Все' "2"
        union all
        select * from temp1''', cnxn) 
        
# КАТЕГОРИИ выпадающий список
    df_2 = pd.read_sql_query(
        '''\
        with temp2 as (
        SELECT
            psl.id "1", psl.name "2"
        FROM catalog.plain_section_list psl
        JOIN opentech.section_new sn ON sn.id = psl.id
            AND sn.id_category_root <> 1070 /* исключил Уценку из фильтра*/
        )
        select 0 "1",  'Все' "2"
        union all
        select * from temp2
        ''', cnxn) 
# Исключить Внутреннее использование и Уцененные товары?
df_3 = pd.DataFrame(data={'1': [0,1], '2': ['ДА','НЕТ']}) 

# Исключить склад НСК Снабжения?
df_4 = pd.DataFrame(data={'1': [0,1], '2': ['ДА','НЕТ']}) 

with psycopg2.connect(dbname=databaseA, user=userA, password=passwordA, host=hostA) as cnxnA:
# ПРОДАКТ выпадающий список
    df_5 = pd.read_sql_query(
        '''\
        with temp3 as (
        select distinct --on(pu.name) 
             cc.id_product_manager "1", pu.name "2" 
        from catalog.categories cc 
        join public.users pu on cc.id_product_manager = pu.id where pu.fired_at is null 
        order by pu.name
        )
        select 0 "1",  'Все' "2"
        union all
        select * from temp3''', cnxnA) 

# прописываем начальные значения 
df_1_val, df_2_val, df_3_val, df_4_val, df_5_val, df_6_val = 0,0,1,1,0,''
# задаем виджеты
description1 = 'Склад:'
description2 = 'Категория:' 
description3 = 'Исключить Внутреннее использование и Уцененные товары?' 
description4 = 'Исключить склад НСК Снабжения?' 
description5 = 'Продакт:' 
description6 = 'Бренд:' 

W1 = widgets.Dropdown( options = [*zip(df_1['2'], df_1.index)], value = df_1_val, description = description1, )
W2 = widgets.Dropdown( options = [*zip(df_2['2'], df_2.index)], value = df_2_val, description = description2, )
W3 = widgets.Dropdown(options=[('НЕ исключать', 1), (description3[:-1], 0)], value=df_3_val,description=description3,)
W4 = widgets.Dropdown(options=[('НЕ исключать', 1), (description4[:-1], 0)], value=df_3_val,description=description4,)
W5 = widgets.Dropdown( options = [*zip(df_5['2'], df_5.index)], value = df_5_val, description = description5, )
W6 = widgets.Text(value=df_6_val,placeholder='Впишите бреннд',description=description6)

def print_param():
    print( f'''\
\t{description1} {df_1.loc[df_1_val,'2']}, ({df_1[df_1['1']==df_1.loc[df_1_val,'1']].index[0]}) \
\t{description2} {df_2.loc[df_2_val,'2']}, ({df_2[df_2['1']==df_2.loc[df_2_val,'1']].index[0]}) \
\t{description3} {df_3.loc[df_3_val,'2']}, ({df_3[df_3['1']==df_3.loc[df_3_val,'1']].index[0]}) \
\t{description4} {df_4.loc[df_4_val,'2']}, ({df_4[df_4['1']==df_4.loc[df_4_val,'1']].index[0]})\
\t{description5} {df_5.loc[df_5_val,'2']}, ({df_5[df_5['1']==df_5.loc[df_5_val,'1']].index[0]})\
\t{description6} {'Бренд не выбран' if df_6_val=='' else df_6_val}\
''' )

# Склад
def W_F_1(W_F_1_dict): 
    global df_1_val
    df_1_val = W_F_1_dict['new']
#     print( df_1_val )
#     print( df_1.loc[df_1_val,'1'] )
#     print_param()

# Категория
def W_F_2(W_F_2_dict): 
    global df_2_val
    df_2_val = W_F_2_dict['new']
#     print( df_2_val )
#     print( df_2.loc[df_2_val,'1'] )
#     print_param()

# Исключить Внутреннее использование и Уцененные товары?
def W_F_3(W_F_3_dict): 
    global df_3_val
    df_3_val = W_F_3_dict['new']
#     print( df_3_val )
#     print( df_3.loc[df_3_val,'1'] )
#     print_param()

# Исключить склад НСК Снабжения?
def W_F_4(W_F_4_dict): 
    global df_4_val
    df_4_val = W_F_4_dict['new']
#     print( df_4_val )
#     print( df_4.loc[df_4_val,'1'] )
#     print_param()

# Продакт
def W_F_5(W_F_5_dict): 
    global df_5_val
    df_5_val = W_F_5_dict['new']
#     print( df_5_val )
#     print( df_5.loc[df_5_val,'1'] )
#     print_param()

# Бренд
def W_F_6(W_F_6_dict): 
    global df_6_val
    df_6_val = W_F_6_dict['new']
#     print( df_6_val )
#     print_param()

def Date_picker_start(date_start):
    global DATE_START_replce
    DATE_START_replce = date_start['new']  # date_start['old'], '->', , date_start
    
def Date_picker_end(date_end):
    global DATE_END_replce
    DATE_END_replce = date_end['new']  # date_end['old'], '->', , date_end

W1.observe(W_F_1, names='value')
W2.observe(W_F_2, names='value')
W3.observe(W_F_3, names='value')
W4.observe(W_F_4, names='value')
W5.observe(W_F_5, names='value')
W6.observe(W_F_6, names='value')

DatePicker1.observe(Date_picker_start, names='value')
# display(DatePicker1)
DatePicker2.observe(Date_picker_end, names='value')
# display(DatePicker2)

# VBox
boxDate = widgets.VBox([ DatePicker1,  DatePicker2 ])
box1 = widgets.VBox([ W1, W2 ])
box2 = widgets.VBox([ W3, W4 ])
box3 = widgets.VBox([ W5, W6 ])
all_VBox_widgets = widgets.HBox([ boxDate,box1,box2,box3 ])

# display
display(all_VBox_widgets)
# print_param();       


Для начала выберите параметры


# Запуск

In [8]:
# print( sql_query_orderstat_site
#                                  .replace('''DATE_START_replce''', f'''{ DATE_START_replce }''') \
#                                  .replace('''DATE_END_replce''', f'''{ DATE_END_replce }''') \
#                                  .replace('''STORAGE_replce''', f'''{ df_1.loc[df_1_val,'1'] }''') \
#                                  .replace('''CATEGORY_replce''', f'''{ df_2.loc[df_2_val,'1'] }''') \
#                                  .replace('''BRAND_replce''', f'''{ df_6_val }''') \
#                                  .replace('''DEL_IN_USE_replce''', f'''{ df_3.loc[df_3_val,'1'] }''') \
#                                  .replace('''DEL_STORAGE_NSK_replce''', f'''{ df_4.loc[df_4_val,'1'] }''') \
#                                  .replace('''PRODUCTMANAGER_replce''', f'''{ df_5.loc[df_5_val,'1'] }''') \

#                                  , cnxnA ) 

# функция кнопки запуска  
def btn_run_function(date):
    #     !pip3 install pyocclient==0.6
    #     !pip install pyocclient # https://pypi.org/project/pyocclient/
    #     import owncloud
    
    public_link = '''https://nextcloud.e2e4.ru/apps/files/?dir=/Public/Отчеты%20(выгрузка%20jupyter%20notebooks)&fileid=2018892'''
    AmazonS3_folder_Public = r'/Public/Отчеты (выгрузка jupyter notebooks)'
    AmazonS3_path_to_file = r'/Среднесуточные складские остатки в разрезе индекса продаж/'
    AmazonS3_file_name = fr'''ССО ({getpass.getuser()}) {DATE_START_replce} - {DATE_END_replce} ({df_5.loc[df_5_val,'2']}) {datetime.datetime.now().strftime("%H:%M:%S")}.xlsx'''
    AmazonS3_full_path = AmazonS3_folder_Public + AmazonS3_path_to_file + AmazonS3_file_name
    #     print('AmazonS3_file_name, AmazonS3_full_path', AmazonS3_file_name, AmazonS3_full_path)
    Amazon_owncloud = owncloud.Client('https://nextcloud.e2e4.ru')
    Amazon_owncloud.login(loginAtlassianjhubAdmin, passwordAtlassianjhubAdmin)
    # Amazon_owncloud.mkdir('testdir')

#     print( f'''Выбранные даты: {DATE_START_replce.strftime("%d.%m.%Y")}    {DATE_END_replce.strftime("%d.%m.%Y")}
#                ({DATE_START_replce})  ({DATE_END_replce}) \n\nИдет выгрузка с БД немного подождите...\n''' )
    #   connect
    with psycopg2.connect(dbname=databaseA, user=userA, password=passwordA, host=hostA) as cnxnA:
    #           подставляем даты 
            df_orderstat_site = pd.read_sql_query(sql_query_orderstat_site \
                                 .replace('''DATE_START_replce''', f'''{ DATE_START_replce }''') \
                                 .replace('''DATE_END_replce''', f'''{ DATE_END_replce }''') \
                                 .replace('''STORAGE_replce''', f'''{ df_1.loc[df_1_val,'1'] }''') \
                                 .replace('''CATEGORY_replce''', f'''{ df_2.loc[df_2_val,'1'] }''') \
                                 .replace('''BRAND_replce''', f'''{ df_6_val }''') \
                                 .replace('''DEL_IN_USE_replce''', f'''{ df_3.loc[df_3_val,'1'] }''') \
                                 .replace('''DEL_STORAGE_NSK_replce''', f'''{ df_4.loc[df_4_val,'1'] }''') \
                                 .replace('''PRODUCTMANAGER_replce''', f'''{ df_5.loc[df_5_val,'1'] }''') \
                                 , cnxnA) 
    # т.е. если  мы не трогали выбор склада, продакта и категории, то просто выводится:
    # по одной строке на индекс, никаких продактов в выдаче не надо
#      df_end_sum = df_orderstat_site[df_orderstat_site.columns[1:]].sum().reset_index()

    if df_1.loc[df_1_val,'1'] + df_2.loc[df_2_val,'1'] + df_5.loc[df_5_val,'1'] == 0:
        df_end_sum = df_orderstat_site[df_orderstat_site.columns[1:]].groupby(by=['Индекс продаж'], dropna=False).sum().reset_index()
    else:
        df_end_sum = pd.concat([df_orderstat_site.iloc[:-1,:].groupby(by=['Продакт','Индекс продаж'], dropna=False).sum().reset_index()
                                ,df_orderstat_site.iloc[-1:,:]])
    print('Выгрузка завершена! \nИдет запись в облако')
    sheet_name1 = 'Лист 1'
    sheet_name2 = 'Лист 2'
    excel_sheets = {
        f'1 - {sheet_name1}' : df_orderstat_site, 
#         f'2 - {sheet_name2}' : df_end_sum, 
        }
    excel_writer = pd.ExcelWriter( temp_tmp + AmazonS3_file_name, engine='xlsxwriter' )
    
    if df_1.loc[df_1_val,'1'] + df_2.loc[df_2_val,'1'] + df_5.loc[df_5_val,'1'] == 0:
        print(
    f'''\
    { colored(sheet_name2, 'red')}\n\
    !! Идет запись в файл (в лист ексель - {sheet_name2}) пожалуйста подождите  \n\
    ''')
        df_end_sum.to_excel(excel_writer, sheet_name=sheet_name2, index=False)
    else:
        for sheet_name in excel_sheets.keys():
            excel_sheets[sheet_name].to_excel(excel_writer, sheet_name=sheet_name, index=False)

    excel_writer.save()
    print('Запись в файл завершена!\n\nИдет запись в облако\n\n') #
    
    try:
        Amazon_owncloud.put_file( 
#           полный путь и там и там!!! 
            AmazonS3_full_path, temp_tmp + AmazonS3_file_name )  
        print( f'Файл записан в облако\n{AmazonS3_full_path} ' )
    except:
        print( f'Возникли проблемы при записи в облако{AmazonS3_full_path}'   )

        os.remove(temp_tmp + AmazonS3_file_name)
    display(HTML(fr"""<a href="{public_link}">Ссылка на nextcloud.e2e4.ru</a>"""))
    # # AttributeError: 'NoneType' object has no attribute 'text' 
    # # https://github.com/owncloud/pyocclient/issues/218 Similar bugs #218, #259. #263 (with a simple fix proposal - a mere workaround)
    # # I have solved this issue within installing version 0.4 of pyocclient pip3 install pyocclient==0.4
    # # Works without any errors - не помогло 
    # https://github.com/owncloud/pyocclient/issues/263 - поменять код в файле либы надо 
    # link_info = Amazon_owncloud.share_file_with_link( AmazonS3_folder_Public )
    # print ("Ссылка: " + link_info.get_link())
    pd.options.display.float_format = '{:.0f}'.format
    display( df_end_sum )

# кнопка запуска
btn = widgets.Button(\
    description='Запуск'\
    ,disabled=False \
    ,button_style='success' \
    ,tooltip='Запуск выгрузки и сохранинение в Excel' \
    ,icon='check'\
    ,layout=Layout(width='33%', height='60px')\
    )
# 'success', 'info', 'warning', 'danger' or ''
display(btn)
btn.on_click(btn_run_function);


Button(button_style='success', description='Запуск', icon='check', layout=Layout(height='60px', width='33%'), …

In [9]:
# class DatePicker(Prints_, MainVariables):
#     #    "change my date on the picker"
#         def change_date_start(self, date_start):
#             global DATE_START_replce
#             DATE_START_replce = date_start['new']  
#     #         print('DATE_START_replce ', DATE_START_replce)

#         def dp_start( self, description = MainVariables().date_picker_description[0], DATE_START_replce = date.today() - relativedelta(months=1) ):
#             date_picker1 = widgets.DatePicker( description = description, disabled = False, value = DATE_START_replce )
#             date_picker1.observe(self.change_date_start, names='value')
#             return date_picker1

#         def change_date_end(self, date_end):
#             global DATE_END_replce
#             DATE_END_replce = date_end['new']  
#     #         print('DATE_END_replce', DATE_END_replce)

#         def dp_end(self, description = MainVariables().date_picker_description[1], DATE_END_replce = date.today() ):
#             date_picker2 = widgets.DatePicker( description = description, disabled = False, value = DATE_END_replce )
#             date_picker2.observe(self.change_date_end, names = 'value')
#             return date_picker2

# def display_all_set_params_widgets():
#     '''display all widgets'''
# #     date start, date end
#     d1 = DatePicker().dp_start( description = MainVariables().date_picker_description[0], DATE_START_replce = DATE_START_replce )
#     d2 = DatePicker().dp_end( description = MainVariables().date_picker_description[1], DATE_END_replce = DATE_END_replce )
# #     brand
#     bp = BrandPicker().brand_picker()
#     # display( d1 ), display( d2 ), display( bp )
    
#     # колонка с датами VBox date_box
#     date_box = widgets.VBox([ 
#         d1
#         ,d2
#         ])
#     box2 = widgets.VBox([ 
#         bp 
#     ])
#     all_VBox_widgets = widgets.HBox([ date_box, box2 ])
#     display(all_VBox_widgets)
# #     print(DATE_START_replce, DATE_END_replce )

# display_all_set_params_widgets()

In [10]:
# ТЕСТЫ